Loading relevant packages

In [209]:
import requests
import json
from json import loads
import pandas as pd
from datetime import datetime
import time
import loading
import util
import processing
import importlib
importlib.reload(loading)
importlib.reload(util)
importlib.reload(processing)

<module 'processing' from '/Users/jeremyelvander/STA-141B-Final-Project/processing.py'>

Using BART API to find relevant station information (Station name, Lattitude & Longitude for mapping)

In [99]:
df = loading.stationLoad(["San Francisco", "Oakland"])


In [100]:
df

,city,name,gtfs_latitude,gtfs_longitude
0,Oakland,12th St. Oakland City Center,37.803768,-122.271450
1,Oakland,Oakland International Airport,37.713238,-122.212191
2,Oakland,MacArthur,37.829065,-122.267040
3,Oakland,Lake Merritt,37.797027,-122.265180
4,Oakland,Rockridge,37.844702,-122.251371
5,Oakland,Coliseum,37.753661,-122.196869
6,Oakland,Fruitvale,37.774836,-122.224175
7,Oakland,19th St. Oakland,37.808350,-122.268602
8,Oakland,West Oakland,37.804872,-122.295140
9,San Francisco,Embarcadero,37.792874,-122.397020


In [145]:

dct = util.dfToDict(df, 'name', ['gtfs_latitude', 'gtfs_longitude'], groupby = 'city')
print(dct)
dct['San Francisco']['Balboa Park']

{'Oakland': {'12th St. Oakland City Center': [37.803768, -122.27145], '19th St. Oakland': [37.80835, -122.268602], 'Coliseum': [37.753661, -122.196869], 'Fruitvale': [37.774836, -122.224175], 'Lake Merritt': [37.797027, -122.26518], 'MacArthur': [37.829065, -122.26704], 'Oakland International Airport': [37.713238, -122.212191], 'Rockridge': [37.844702, -122.251371], 'West Oakland': [37.804872, -122.29514]}, 'San Francisco': {'16th St. Mission': [37.765062, -122.419694], '24th St. Mission': [37.75247, -122.418143], 'Balboa Park': [37.721585, -122.447506], 'Civic Center/UN Plaza': [37.779732, -122.414123], 'Embarcadero': [37.792874, -122.39702], 'Glen Park': [37.733064, -122.433817], 'Montgomery St.': [37.789405, -122.401066], 'Powell St.': [37.784471, -122.407974]}}


[37.721585, -122.447506]

In [102]:
testRow = df.iloc[0]
testRow['city']
city = dct[testRow['city']]
# for station in city:
#     print(city[station])
stationCheck = util.inStationRadius(dct, testRow, radius = 0.5)
print(stationCheck)

{'12th St. Oakland City Center': 0.0, '19th St. Oakland': 0.25940999843877033, 'Lake Merritt': 0.4995130704552501}


In [159]:
secretKey = '1ef8dner8br348ez190z3z188a6pm1yluwh2wcj4xjjnlvupe'
key = '523e7ewewudxb1frvbmdftwdt'

data = loading.loadCrime("SF", key, secretKey, dct['San Francisco']['Balboa Park'],
                         0.25, 2021, 4, 0)

data1 = loading.loadCrime("SF", key, secretKey, dct['San Francisco']['Balboa Park'],
                         0.25, 2021, 2, 0)

data2 = loading.loadCrime("SF", key, secretKey, dct['San Francisco']['Balboa Park'],
                         0.25, 2021, 2, 2)

In [166]:
# import pprint

# pprint.pprint(data)

# pprint.pprint(data1)

# pprint.pprint(data2)

# # for dction in data:
# #     if dction['incident_category'] == "Malicious Mischief":
# #         pprint.pprint(dction)

data[0]

{'incident_datetime': '2021-01-05T14:30:00.000',
 'incident_category': 'Larceny Theft',
 'latitude': '37.72175241158197',
 'longitude': '-122.44929574336275',
 'point': {'type': 'Point',
  'coordinates': [-122.44929574336275, 37.72175241158197]}}

In [198]:
import pprint
def func(function, params):
    return function(*params)

test = func(loading.loadCrime, ["SF", dct['San Francisco']['Balboa Park'],
                         0.25, 2021, 2, 0, key, secretKey])

In [202]:
test

[{'incident_datetime': '2021-01-05T14:30:00.000',
  'incident_category': 'Larceny Theft',
  'point': {'type': 'Point',
   'coordinates': [-122.44929574336275, 37.72175241158197]}},
 {'incident_datetime': '2021-01-07T07:47:00.000',
  'incident_category': 'Motor Vehicle Theft',
  'point': {'type': 'Point',
   'coordinates': [-122.44756206258599, 37.71948111883161]}}]

In [194]:
queries = [['SF', dct['San Francisco']['Balboa Park'], 0.25, 2021, 10, 0],
           ['SF', dct['San Francisco']['Embarcadero'], 0.25, 2021, 10, 0]]
queries2 = [['OAK', dct['Oakland']['Fruitvale'], 0.25, 2021, 10, 0],
            ["OAK", dct['Oakland']['MacArthur'], 0.25, 2021, 10, 0]]
           

testFrame = processing.parallelProcess(3, loading.loadCrime, queries, 
                                       'load', True, key, secretKey)
testFrame2 = processing.parallelProcess(3, loading.loadCrime, queries2, 
                                       'load', True, key, secretKey)

Time taken to process (seconds): 5.593416929244995
Time taken to process (seconds): 5.461392879486084


In [195]:
renames = {'incident_datetime': 'datetime',
           'point': 'location', 'incident_category':'crime',
           'crimetype': 'crime'}
rename = processing.parallelProcess(3, util.Rename, [testFrame, testFrame2],
                                                   'other', False, renames)
testFrameFinal = pd.concat(rename, ignore_index = True)
testFrameFinal

Time taken to process (seconds): 0.0064771175384521484


,datetime,crime,location
0,2021-01-05T14:30:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4492957..."
1,2021-01-07T07:47:00.000,Motor Vehicle Theft,"{'type': 'Point', 'coordinates': [-122.4475620..."
2,2021-01-07T14:00:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4492791..."
3,2021-01-07T16:20:00.000,Malicious Mischief,"{'type': 'Point', 'coordinates': [-122.4466086..."
4,2021-01-08T17:05:00.000,Other Miscellaneous,"{'type': 'Point', 'coordinates': [-122.4451278..."
5,2021-01-10T20:00:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4508206..."
6,2021-01-12T20:48:00.000,Weapons Offense,"{'type': 'Point', 'coordinates': [-122.4477915..."
7,2021-01-13T23:30:00.000,Lost Property,"{'type': 'Point', 'coordinates': [-122.4508206..."
8,2021-01-14T20:30:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4509272..."
9,2021-01-31T08:00:00.000,Other Miscellaneous,"{'type': 'Point', 'coordinates': [-122.4454847..."


In [211]:
queries = [['SF', dct['San Francisco']['Balboa Park'], 0.25, 2021, 10, 0],
           ['SF', dct['San Francisco']['Embarcadero'], 0.25, 2021, 10, 0]]

newCols = [{'city': "San Francisco", "station":"Balboa Park"},
           {"city": "San Francisco", "station":"Embarcadero"}]

testFrame = processing.parallelProcess(3, loading.loadCrime, queries, 
                                       'load', True, [util.addColumnsJSON] * len(queries),
                                       newCols, key, secretKey)

testFrame

Time taken to process (seconds): 4.689861297607422


,incident_datetime,incident_category,point,city,station
0,2021-01-05T14:30:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4492957...",San Francisco,Embarcadero
1,2021-01-07T07:47:00.000,Motor Vehicle Theft,"{'type': 'Point', 'coordinates': [-122.4475620...",San Francisco,Embarcadero
2,2021-01-07T14:00:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4492791...",San Francisco,Embarcadero
3,2021-01-07T16:20:00.000,Malicious Mischief,"{'type': 'Point', 'coordinates': [-122.4466086...",San Francisco,Embarcadero
4,2021-01-08T17:05:00.000,Other Miscellaneous,"{'type': 'Point', 'coordinates': [-122.4451278...",San Francisco,Embarcadero
5,2021-01-10T20:00:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4508206...",San Francisco,Embarcadero
6,2021-01-12T20:48:00.000,Weapons Offense,"{'type': 'Point', 'coordinates': [-122.4477915...",San Francisco,Embarcadero
7,2021-01-13T23:30:00.000,Lost Property,"{'type': 'Point', 'coordinates': [-122.4508206...",San Francisco,Embarcadero
8,2021-01-14T20:30:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4509272...",San Francisco,Embarcadero
9,2021-01-31T08:00:00.000,Other Miscellaneous,"{'type': 'Point', 'coordinates': [-122.4454847...",San Francisco,Embarcadero


In [212]:
queries = [['SF', dct['San Francisco']['Balboa Park'], 0.25, 2021, 10, 0],
           ['SF', dct['San Francisco']['Embarcadero'], 0.25, 2021, 10, 0]]

# Metadata: One dictionary for each station, matching the order of queries
newCols = [{'city': "San Francisco", "station": "Balboa Park"},
           {'city': "San Francisco", "station": "Embarcadero"}]

# Pass queries and corresponding newCols metadata to the parallelProcess function
testFrame = processing.parallelProcess(3, loading.loadCrime, queries, 
                                       'load', True, 
                                       [util.addColumnsJSON] * len(queries),  # Same function for each
                                       newCols, key, secretKey)

testFrame

Time taken to process (seconds): 5.451158046722412


,incident_datetime,incident_category,point,city,station
0,2021-01-05T14:30:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4492957...",San Francisco,Embarcadero
1,2021-01-07T07:47:00.000,Motor Vehicle Theft,"{'type': 'Point', 'coordinates': [-122.4475620...",San Francisco,Embarcadero
2,2021-01-07T14:00:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4492791...",San Francisco,Embarcadero
3,2021-01-07T16:20:00.000,Malicious Mischief,"{'type': 'Point', 'coordinates': [-122.4466086...",San Francisco,Embarcadero
4,2021-01-08T17:05:00.000,Other Miscellaneous,"{'type': 'Point', 'coordinates': [-122.4451278...",San Francisco,Embarcadero
5,2021-01-10T20:00:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4508206...",San Francisco,Embarcadero
6,2021-01-12T20:48:00.000,Weapons Offense,"{'type': 'Point', 'coordinates': [-122.4477915...",San Francisco,Embarcadero
7,2021-01-13T23:30:00.000,Lost Property,"{'type': 'Point', 'coordinates': [-122.4508206...",San Francisco,Embarcadero
8,2021-01-14T20:30:00.000,Larceny Theft,"{'type': 'Point', 'coordinates': [-122.4509272...",San Francisco,Embarcadero
9,2021-01-31T08:00:00.000,Other Miscellaneous,"{'type': 'Point', 'coordinates': [-122.4454847...",San Francisco,Embarcadero
